In [1]:
#installing Kaggle
!pip install kaggle


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 23.9 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.12-py3-none-any.whl size=102969 sha256=6433206665d3402478b66b29b49b446eea2f3a4647769682fd048b78e578a836
  Stored in directory: /home/gitpod/.cache/pip/wheels/a0/87/83/f9893bb1660a03ca5808b7429e77808120f091569651323dd1
Successfully built kaggle


In [2]:
#downloading kaggle dataset for cherry leaves
!kaggle datasets download -d codeinstitute/cherry-leaves


Dataset URL: https://www.kaggle.com/datasets/codeinstitute/cherry-leaves
License(s): unknown
 91%|██████████████████████████████████▌   | 50.0M/55.0M [00:01<00:00, 46.3MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:01<00:00, 39.1MB/s]


In [4]:
#unzipping the file
import zipfile

with zipfile.ZipFile("cherry-leaves.zip", "r") as zip_ref:
    zip_ref.extractall("cherry-leaves")


In [18]:
import os
cwd = os.getcwd()
print("Current working directory:", cwd)


Current working directory: /workspace/milestone-project5-mildew-detection-in-cherry-leaves/jupyter_notebooks


In [4]:
from PIL import Image
import os

# Define directory paths
dataset_dirs = ["./cherry-leaves/cherry-leaves-folder/healthy", "./cherry-leaves/cherry-leaves-folder/powdery_mildew"]
output_dir = "./clean_data"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to resize images and save them
def resize_and_save_image(image_path, output_path, new_size=(50, 50)):
    # Load image
    image = Image.open(image_path)
    
    # Resize image
    resized_image = image.resize(new_size)
    
    # Save resized image
    resized_image.save(output_path)

# Iterate through dataset directories
for dataset_dir in dataset_dirs:
    # Get category name from directory
    category = os.path.basename(dataset_dir)
    
    # Output directory for the current category
    output_category_dir = os.path.join(output_dir, category)
    os.makedirs(output_category_dir, exist_ok=True)
    
    # Iterate through files in the dataset directory
    for file in os.listdir(dataset_dir):
        if file.lower().endswith((".jpeg", ".jpg")):
            # Input and output paths
            input_path = os.path.join(dataset_dir, file)
            output_path = os.path.join(output_category_dir, file)
            
            # Resize image and save
            resize_and_save_image(input_path, output_path)


In [7]:
from PIL import Image
import os
import numpy as np

# Define directory paths
dataset_dirs = ["./cherry-leaves/cherry-leaves-folder/healthy", "./cherry-leaves/cherry-leaves-folder/powdery_mildew"]

# Function to compute color histogram features
def compute_color_histogram(image):
    # Convert image to HSV color space
    hsv = image.convert("HSV")
    
    # Compute histograms for each channel
    hist_hue = hsv.histogram(mask=None, extrema=None)
    hist_saturation = hsv.histogram(mask=None, extrema=None)
    hist_value = hsv.histogram(mask=None, extrema=None)
    
    # Normalize histograms
    hist_hue = np.array(hist_hue) / np.sum(hist_hue)
    hist_saturation = np.array(hist_saturation) / np.sum(hist_saturation)
    hist_value = np.array(hist_value) / np.sum(hist_value)
    
    # Concatenate histograms into a single feature vector
    hist_features = np.concatenate((hist_hue, hist_saturation, hist_value))
    
    return hist_features


# Iterate through dataset directories
for dataset_dir in dataset_dirs:
    category = os.path.basename(dataset_dir)
    
    # Iterate through files in the dataset directory
    for file in os.listdir(dataset_dir):
        if file.lower().endswith((".jpeg", ".jpg")):
            # Load image
            image = Image.open(os.path.join(dataset_dir, file))
            
            # Compute color histogram features
            color_hist_features = compute_color_histogram(image)
            
            # Determine if the leaf is healthy or powdery mildew based on color histogram
            # Example: Apply a simple threshold
            if color_hist_features[0] > 0.5:  # Assuming the first feature represents hue histogram
                print(f"Leaf {file} in category {category} is healthy")
            else:
                print(f"Leaf {file} in category {category} has powdery mildew")
# Define directory paths for processed data
processed_data_dir = "./processed_data"
healthy_data_dir = os.path.join(processed_data_dir, "healthy")
mildew_data_dir = os.path.join(processed_data_dir, "powdery_mildew")

# Create directories if they don't exist
os.makedirs(healthy_data_dir, exist_ok=True)
os.makedirs(mildew_data_dir, exist_ok=True)

# Iterate through dataset directories
for dataset_dir in dataset_dirs:
    category = os.path.basename(dataset_dir)
    
    # Determine output directory based on category
    output_category_dir = healthy_data_dir if category == "healthy" else mildew_data_dir
    
    # Iterate through files in the dataset directory
    for file in os.listdir(dataset_dir):
        if file.lower().endswith((".jpeg", ".jpg")):
            # Load image
            image = Image.open(os.path.join(dataset_dir, file))
            
            # Compute color histogram features
            color_hist_features = compute_color_histogram(image)
            
            # Save processed data (features and label) as numpy array
            np.save(os.path.join(output_category_dir, file.replace(".jpg", "").replace(".jpeg", "") + ".npy"), color_hist_features)

TypeError: histogram() got an unexpected keyword argument 'source'